In [0]:
--Calculate 7 Day Moving Average
select
  ticker,
  trade_date,
  close,
  avg(close) over (
    partition by ticker
    order by trade_date
    rows between 6 preceding and current row
    ) as moving_average_7_day
from
  analytics.stock_prices
  where trade_date >= current_date() - interval 30 days;

--Calculate Strict 7 Day Moving Average like a Trading Platform

with numbered as (
  select *,
    row_number() over (
      partition by ticker
      order by trade_date
    ) as rn
  from analytics.stock_prices
)

select
  ticker,
  trade_date,
  close,
  case
    when rn >= 7 then 
      avg(close) over (
      partition by ticker
      order by trade_date
      rows between 6 preceding and current row
    )
  end as 7day_moving_average
  from numbered;

  --Calculate 30 Day Rolling Volatility

  with base as (
    select
      ticker,
      trade_date,
      close,
      lag(close) over (
        partition by ticker
        order by trade_date
      ) as prev_close
    from analytics.stock_prices
  ),

  returns as (
    select
      ticker,
      trade_date,
      close,
      (close - prev_close) / prev_close as daily_return
    from base
    where prev_close is not null
  ),

  volatility as (
    select
      ticker,
      trade_date,
      close,
      daily_return,
      stddev_samp(daily_return) over (
        partition by ticker
        order by trade_date
        rows between 29 preceding and current row
      )* 100 as daily_volatility
    from returns
  )

  select * 
  from volatility
  where daily_volatility is not null;


  --Calculate Strict 30 Day Rolling Volatility like a Trading Platform

with base as (
    select
      ticker,
      trade_date,
      close,
      lag(close) over (
        partition by ticker
        order by trade_date
      ) as prev_close
    from analytics.stock_prices
  ),

returns as (
    select
      ticker,
      trade_date,
      close,
      (close - prev_close) / prev_close as daily_return
    from base
    where prev_close is not null
  ),

numbered_returns as (
    select *,
    row_number() over (
      partition by ticker
      order by trade_date
    ) as rn_return
    from returns
  ),

volatility as (
    select
      ticker,
      trade_date,
      close,
      daily_return,
      case 
        when rn_return >= 30 then 
          stddev_samp(daily_return) over (
            partition by ticker
            order by trade_date
            rows between 29 preceding and current row
          ) * 100
        end as daily_volatility
      from numbered_returns
  )

select *
from volatility;


--Rolling Max Drawdown

with base as (
  select 
    ticker,
    trade_date,
    close,
    max(close) over (
      partition by ticker
      order by trade_date
      rows between unbounded preceding and current row
    ) as rolling_max
  from analytics.stock_prices
)

select
  ticker,
  trade_date,
  close,
  rolling_max,
  (close - rolling_max) / rolling_max * 100 as drawdown
from base;


--Volume average vs current volume spike detection

with filtered as (
  select *
  from analytics.stock_prices
),

volume_avg as (
  select
    ticker,
    trade_date,
    volume,
    avg(volume) over (
      partition by ticker
      order by trade_date
      rows between 19 preceding and current row
    ) as avg_volume_20
  from filtered
),

volume_spike as (
  select
    ticker,
    trade_date,
    volume,
    avg_volume_20,
    case
      when volume > 2 * avg_volume_20 then 1
      else 0
    end as volume_spike_flag
  from volume_avg
)

select *
from volume_spike
where volume_spike_flag = 1;

--Volume average vs current volume spike detection (strict 20 day average)

with numbered as (
  select *,
  row_number() over (
    partition by ticker
    order by trade_date
  ) as rn
from analytics.stock_prices
),

volume_avg as (
  select
    ticker,
    trade_date,
    volume,
    rn,
    case 
      when rn >= 20 then 
      avg(volume) over (
        partition by ticker
        order by trade_date
        rows between 19 preceding and current row
      )
    end as avg_volume_20
  from numbered
),

volume_spike as (
  select 
    ticker,
    trade_date,
    volume,
    avg_volume_20,
    volume / avg_volume_20 as volume_spike_ratio
  from volume_avg
)

select *
from volume_spike
where volume_spike_ratio > 1.85
and avg_volume_20 is not null;

--Volume Spike Thresholds to figure out Significant Spike Ratios
select
  percentile_cont(0.5) within group (order by volume_spike_ratio) as median,
  percentile_cont(0.75) within group (order by volume_spike_ratio) as p75,
  percentile_cont(0.9) within group (order by volume_spike_ratio) as p90,
  percentile_cont(0.95) within group (order by volume_spike_ratio) as p95
from volume_spike
where avg_volume_20 is not null;
  

 
